In [ ]:
def get_optimal_route(start_location, end_location):
    # Copy the rewards matrix to new Matrix
    rewards_new = np.copy(rewards)
    
    # Get the ending state corresponding to the ending location as given
    ending_state = location_to_state[end_location]
    
    # With the above information automatically set the priority of the given ending 
    # state to the highest one
    rewards_new[ending_state,ending_state] = 999

    # -----------Q-Learning algorithm-----------
   
    # Initializing Q-Values
    Q = np.array(np.zeros([9,9]))

    # Q-Learning process
    for i in range(1000):
        # Pick up a state randomly
        current_state = np.random.randint(0,9)
        # For traversing through the neighbor locations in the maze
        playable_actions = []
        # Iterate through the new rewards matrix and get the actions > 0
        for j in range(9):
            if rewards_new[current_state,j] > 0:
                playable_actions.append(j)
        # Pick an action randomly from the list of playable actions  
        # leading us to the next state
        next_state = np.random.choice(playable_actions)
        # Compute the temporal difference
        # The action here exactly refers to going to the next state
        TD = rewards_new[current_state,next_state] + gamma * Q[next_state, np.argmax(Q[next_state,])] - Q[current_state,next_state]
        # Update the Q-Value using the Bellman equation
        Q[current_state,next_state] += alpha * TD

In [ ]:
import math


class Robot:
    def __init__(self, x, y, game):
        self.x, self.y = x, y
        self.game = game
    
    @property
    def coords(self): return self.x, self.y
    
    def __repr__(self):
        return 'R'
    
    def set_state(self, x, y):
        self.game[self.y][self.x] = 0
        self.x, self.y = x, y
        self.game[y][x] = self
        

class Environment:
    def __repr__(self):
        return '\n'.join(map(lambda row: ' '.join(map(lambda x: x.__repr__(), row)), self.state))
    
    def get_string(self):
        string = ''
        for i, row in enumerate(self.state):
            string += ' | '.join(f'{ele}: {self.get_score((j, i)) if (j, i) in self.get_actions() else 0}' for j, ele in enumerate(row)) + '\n'
        return string
            
    def __init__(self, size, illegal, start, goal):
        self.size = size
        self.start = start
        self.goal = goal
        self.state = [[0] * size for _ in range(size)]
        self.prev = []  # (state, turn)
        
        self.illegal = illegal
        self.robot = Robot(*start, self.state)
        self.robot.set_state(*start)
    
    def set_goal(self):
        self.state[self.goal[1]][self.goal[0]] = 1 << 16
        
    def reset(self):
        self.state = [[0] * size for _ in range(size)]
        self.robot.set_state(*self.start)
        self.set_goal()
        
    def get_pos(self, x, y):
        if 0 <= x < self.size and 0 <= y < self.size:
            return self.state[y][x]
        return None
    
    def step(self):
        action = max(self.get_actions(), key=self.get_score)
        self.robot.set_state(*action)
        self.prev.append(())
        
    def get_score(self, action):
        if action == self.robot.coords and action != self.goal:
            return 0
        
        if (self.robot.coords, action) not in self.illegal:
            return 1 if action != self.goal else 1 << 16
        return -1
    
    def get_actions(self):
        return [a for a in [
            self.robot.coords,
            (self.robot.x - 1, self.robot.y),
            (self.robot.x, self.robot.y - 1),
            (self.robot.x + 1, self.robot.y),
            (self.robot.x, self.robot.y + 1)
        ] if self.get_pos(*a) is not None]

illegal = [
    ((0, 1), (0, 0)),
    ((0, 1), (1, 1)),
    ((2, 1), (1, 1)),
    ((2, 1), (2, 2))
    ]

for a, b in illegal[:]:
    illegal.append((b, a))
    
env = Environment(3, illegal, (0, 1), (2, 1))

print(env.get_string())

for _ in range(10):
    env.step()
    print(env)

In [151]:
from random import choice, sample, randint
import numpy as np
from tkinter import Tk, Canvas
from math import log10


class App:
    @staticmethod
    def grid_bounding(r, c):
        c, r = c * 100, r * 100
        return [(c + 100 * a, r + 100 * b) for a in (0, 1) for b in (0, 1)]

    def flatten(self, p):
        return (lambda r, c: self.size * r + c)(*p)
    
    def expand(self, k):
        return k // self.size, k % self.size
    
    @classmethod
    def random_start(cls, size, blocks=0.2, walls=0.2, start=None, goal=None, **kwargs):
        def random_inc(i):
            if i == 0:
                return 1
            if i == size - 1:
                return i - 1
            return i + choice((-1, 1))
            
        def random_adjacent(r, c):
            if randint(0, 1):
                return random_inc(r), c
            return r, random_inc(c)
        
        if start is None:
            start = 0, 0
        elif start is True:
            start = randint(0, size - 1), randint(0, size - 1)
            
        if goal is None:
            goal = size - 1, size - 1
        elif goal is True:
            goal = randint(0, size - 1), randint(0, size - 1)
            
        s2 = size * size
        unique = [(a, b) for a in range(size) for b in range(size) if (a, b) not in (start, goal)]
        
        blocks = sample(unique, int(blocks * s2))
        walls = [(p, random_adjacent(*p)) for p in sample(unique, int(walls * s2))]
        
        return cls(size, start=start, goal=goal, blocks=blocks, walls=walls, **kwargs)
    
    def __init__(self, size, start=None, goal=None, blocks=None, walls=None,
                 rounds=1000, decay=0.75, step_size=1):
        self.size = size
        self.s2 = size * size 
        
        self.rounds = rounds
        self.decay = decay
        self.step = 1

        # creating window
        self.dim = size * 100
        
        self.root = Tk()
        self.canvas = Canvas(self.root, width=self.dim, height=self.dim)

        self.root.geometry('{0}x{0}'.format(self.dim))
        self.canvas.pack()

        self.mouse_down = False
        self.mouse_start = None
        self.canvas.bind('<Button-3>', lambda e: self.iterate_pos((e.y // 100, e.x // 100)))
        self.canvas.bind('<ButtonPress-1>', lambda e: self.toggle_press(e.y // 100, e.x // 100))
        self.canvas.bind('<ButtonRelease-1>', lambda e: self.toggle_press(e.y // 100, e.x // 100))

        self.root.bind('<Return>', lambda *_: self.model())
        self.root.bind('<BackSpace>', lambda *_: self.canvas.delete('path'))
        
        self.showing_numbers = False
        self.root.bind('<KeyPress>', lambda e: self.key_handler(e.keysym, True))
        self.root.bind('<KeyRelease>', lambda e: self.key_handler(e.keysym, False))
        
        self.showing_moves = False
                
        self.start = start
        self.goal = goal

        self.walls = [] if walls is None else walls
        self.blocks = [] if blocks is None else blocks
        self.legality = np.zeros
        
    def key_handler(self, key, boolean):
        if key == 'space':
            self.showing_numbers = boolean
            self.draw_nums() if boolean and not self.showing_numbers else self.canvas.delete('numbers')
            
        elif key == 'm':
            self.showing_moves = boolean
            self.show_moves((self.root.winfo_pointery() - self.root.winfo_rooty()) // 100, (self.root.winfo_pointerx() - self.root.winfo_rootx()) // 100) if boolean else self.canvas.delete('moves')
            
    def show_moves(self, r, c):
        for j, legal in enumerate(self.get_info()[2][r * self.size + c]):  # bad stopgap, keep running log
            if legal:
                r1, c1 = self.expand(j)
                self.canvas.create_rectangle(c1 * 100, r1 * 100, c1 * 100 + 100, r1 * 100 + 100,
                                             fill='blue', tags=('moves',))
        
    def run(self, auto):
        self.root.after(10, self.draw_grid)
        if auto and None not in (self.start, self.goal):
            self.root.after(100, self.model)
        self.root.mainloop()
        
    def draw_nums(self):
        for r in range(self.size):
            for c in range(self.size):
                i = r * self.size + c
                self.canvas.create_text(c * 100 + 50, r * 100 + 50, text=str(i), font=('Arial', 20), fill='#444444', tags=('numbers', str(i)))
                
    def draw_grid(self):
        for r in range(self.size):
            for c in range(self.size):
                pos = r, c
                if pos in self.blocks:
                    color = 'grey'
                elif pos == self.goal:
                    color = 'green'
                elif pos == self.start:
                    color = 'yellow'
                else:
                    color = 'white'
                    
                x0, y0 = c * 100, r * 100
                self.canvas.create_rectangle(x0, y0, x0 + 100, y0 + 100,
                                             fill=color, width=5, tags=('rc_%sx%s' % pos,))
                
        for a, b in self.walls[:]:
            self.walls.append((b, a))
            
            a_p, b_p = self.grid_bounding(*a), self.grid_bounding(*b)
            self.canvas.create_line(*(lambda a, b: a + b)(*tuple(set(a_p).intersection(b_p))),
                                    fill='red', width=5, tags=(f'wall_{a}x{b}', f'wall_{b}x{a}'))
            
    def toggle_press(self, r, c):
        self.mouse_down = not self.mouse_down
        
        if self.mouse_down:
            self.mouse_start = r, c
        elif (r, c) == self.mouse_start:
            self.set_block(r, c)
        else:
            self.set_wall(self.mouse_start, (r, c))
            self.mouse_start = None

    def set_block(self, r, c, override=False):
        if (r, c) in self.blocks:
            self.canvas.itemconfigure(f'rc_{r}x{c}', fill='white')
            self.blocks.remove((r, c))
            return

        self.canvas.itemconfigure(f'rc_{r}x{c}', fill='grey')
        self.blocks.append((r, c))

    def iterate_pos(self, p):  # null -> start -> goal -> null
        if p in self.blocks:
            return

        color = 'yellow'
        if self.goal is None and p == self.start:
            self.start, self.goal, color = None, p, 'green'
        elif p == self.goal:
            self.goal, color = None, 'white'
        elif self.start is None:
            self.start = p
        else:
            print('cannot have more than 1 start position at any given time')
            return

        self.canvas.itemconfigure('rc_%sx%s' % p, fill=color)
        
    def set_wall(self, a, b):
        if (a, b) in self.walls:
            self.walls.remove((a, b))
            self.walls.remove((b, a))

            self.canvas.delete(f'wall_{a}x{b}')
            return

        self.walls.append((a, b))
        self.walls.append((b, a))

        a_p, b_p = self.grid_bounding(*a), self.grid_bounding(*b)
        intersection = tuple(set(a_p).intersection(b_p))

        if len(intersection) != 2:
            print(f'{a} and {b} were not adjacent')
            return

        p0, p1 = intersection
        self.canvas.create_line(*p0, *p1, fill='red', width=5, tags=(f'wall_{a}x{b}', f'wall_{b}x{a}'))

    def get_info(self):        
        def legal(i, k):
            if not (0 <= k < self.s2):
                return False
            
            if abs(i - k) == 1:
                return i // self.size == k // self.size
            return True
        
        legality = []
        for i in range(self.s2):
            moves = list(filter(lambda k: legal(i, k), (i, i - self.size, i + self.size, i - 1, i + 1)))
            legality.append([int(k in moves) for k in range(self.s2)])
            
        legality = np.array(legality)
    
        goal = self.flatten(self.goal)
        legality[goal, goal] = 1 << 8
        
        for wall in self.walls:
            legality[tuple(map(self.flatten, wall))] = 0
            
        for block in map(self.flatten, self.blocks):
            for row in legality:
                row[block] = 0
        
        return self.flatten(self.start), goal, legality
    
    def model(self):
        start, goal, legality = self.get_info()
        actions = list(range(self.s2))

        Q = np.zeros([self.s2, self.s2])

        l = int(log10(self.rounds)) + 1
        for rnd in range(self.rounds):
            if rnd % 100 == 99:
                print(f'\r{rnd + 1:>{l}} / {self.rounds}', end='')
            state = choice(actions)
            action = choice(list(filter(lambda action: legality[state, action] > 0, actions)))

            Q[state, action] += self.step * (legality[state, action] + self.decay * Q[action, np.argmax(Q[action,])] - Q[state, action])  # update Q-values with the TD
                    
        print('\nfinished training')
        
        state, path = start, [start]
        for _ in range(self.s2):
            state = np.argmax(Q[state,])
            path.append(state)
            if state == goal:  # goal
                break
        else:
            print('path not found')

        self.path_display(path)
    
    def path_display(self, path):
        self.canvas.delete('path')
        
        points = [(lambda b: [sum(c) / 4 for c in b])(zip(*self.grid_bounding(*self.expand(point)))) for point in path]
        for i, point in enumerate(points[1:]):
            self.canvas.create_line(*points[i], *points[i + 1], fill='blue', width=3, tags='path')

In [13]:
from random import choice, sample, randint
import numpy as np
from tkinter import Tk, Canvas
from math import log10
from time import perf_counter
from scipy import interpolate


SIZE = 100
GOAL = 9

WALL_WIDTH = 1

BG = 'white'
BLOCK_COLOR = '#333333'
WALL_COLOR = BLOCK_COLOR
GOAL_COLOR = 'green'
START_COLOR = 'red'
GRID_COLOR = ''
WALK_COLOR = 'blue'


class Legality:
    def adjacent(self, k):
        r, c = k // self.row, k % self.row
        if r != 0:
            yield k - self.row
        if r != self.row - 1:
            yield k + self.row
        if c != 0:
            yield k - 1
        if c != self.row - 1:
            yield k + 1
        yield k
            
    def legal(self, state, action):
        if action == state:
            return True

        if action in self.blocks:
            return False

        if (state, action) in self.walls or (action, state) in self.walls:
            return False

        if not (0 <= action < self.size):
            return False

        if abs(state - action) == 1:
            return state // self.row == action // self.row
        else:
            return state % self.row == action % self.row

    def __getitem__(self, item):
        return self.arr[item]

    def __setitem__(self, key, value):
        self.arr[key] = value

    def __init__(self, size, goal, walls, blocks):
        self.row = size
        self.size = size * size

        self.walls, self.blocks = walls, blocks
        self.goal = goal

        self.arr = []
        for i in range(self.size):
            self.arr.append([int(self.legal(i, k)) for k in range(self.size)])

        self.arr = np.array(self.arr)
        if goal is not None:
            self[goal, goal] = GOAL

    def moves(self, i):
        return [k for k in self.adjacent(i) if self.legal(i, k)]

    def moves_2d(self, r, c):
        i = r * self.row + c
        return [k for k in self.adjacent(i) if self.legal(i, k)]

    def change_goal(self, goal):
        if self.goal:
            self[self.goal, self.goal] = 1
            
        self[goal, goal] = GOAL
        self.goal = goal

        
class App2:
    @staticmethod
    def grid_bounding(r, c):
        c, r = c * SIZE, r * SIZE
        return [(c + SIZE * a, r + SIZE * b) for a in (0, 1) for b in (0, 1)]

    @staticmethod
    def scale_down(e):
        return e.y // SIZE, e.x // SIZE

    def grid_bounding_flat(self, i):
        r, c = map(lambda x: x * SIZE, self.expand(i))
        return [(c + SIZE * a, r + SIZE * b) for a in (0, 1) for b in (0, 1)]

    def flatten(self, *p):
        return (lambda r, c: self.size * r + c)(*(p if len(p) > 1 else p[0]))

    def expand(self, k):
        return k // self.size, k % self.size

    @classmethod
    def random_start(cls, size, blocks=0.2, walls=0.2, start=None, goal=None, **kwargs):
        s2 = size * size

        def random_adjacent(k):
            choices = []
            r, c = k // size, k % size
            if r != 0:
                choices.append(-size)
            if r != size - 1:
                choices.append(size)
            if c != 0:
                choices.append(-1)
            if c != size - 1:
                choices.append(1)
            return k + choice(choices)

        if start is None:
            start = 0
        elif start is True:
            start = randint(0, s2 - 1)

        if goal is None:
            goal = s2 - 1
        elif goal is True:
            goal = randint(0, s2 - 1)

        unique = [i for i in range(s2) if i not in (start, goal)]

        blocks = sample(unique, int(blocks * s2))
        walls = [(k, random_adjacent(k)) for k in sample(unique, int(walls * s2))]

        return cls(size, start=start, goal=goal, blocks=blocks, walls=walls, **kwargs)

    def __init__(self, size, start=None, goal=None, blocks=None, walls=None,
                 rounds=1000, decay=0.75, step=1):
        self.size = size
        self.s2 = size * size

        self.rounds = rounds
        self.decay = decay
        self.step = step

        # creating window
        self.dim = size * SIZE

        self.root = Tk()
        self.canvas = Canvas(self.root, width=self.dim, height=self.dim)

        self.root.geometry('{0}x{0}'.format(self.dim))
        self.canvas.pack()

        self.mouse_down = False
        self.mouse_start = None
        self.canvas.bind('<Button-3>', lambda e: self.iterate_pos(self.flatten(self.scale_down(e))))
        self.canvas.bind('<ButtonPress-1>', lambda e: self.toggle_press(*self.scale_down(e)))
        self.canvas.bind('<ButtonRelease-1>', lambda e: self.toggle_press(*self.scale_down(e)))
        self.canvas.bind('<Motion>', lambda e: self.reset_drawings())

        self.root.bind('<Return>', lambda *_: self.model())
        self.root.bind('<BackSpace>', lambda *_: self.canvas.delete('path', 'Walker'))

        self.showing_numbers = False
        self.root.bind('<KeyPress>', lambda e: self.key_handler(e.keysym, True))
        self.root.bind('<KeyRelease>', lambda e: self.key_handler(e.keysym, False))

        self.showing_moves = False

        self.start = start
        self.goal = goal

        self.walls = [] if walls is None else walls
        self.blocks = [] if blocks is None else blocks
        
        self.legality = Legality(size, goal=goal, walls=self.walls, blocks=self.blocks)

    def reset_drawings(self):
        if self.showing_moves:
            self.canvas.delete('moves')
            self.show_moves()
            
    def key_handler(self, key, boolean):
        if key == 'space':
            self.showing_numbers = boolean
            self.draw_nums() if boolean and self.showing_numbers else self.canvas.delete('numbers')

        elif key == 'm':
            self.showing_moves = boolean
            if boolean:
                self.show_moves()
            else:
                self.canvas.delete('moves')

    def show_moves(self):
        r, c = (max(0, self.root.winfo_pointery() - self.root.winfo_rooty()) // SIZE,
                max(0, self.root.winfo_pointerx() - self.root.winfo_rootx()) // SIZE)
        for j in self.legality.moves_2d(r, c):
            r1, c1 = self.expand(j)
            self.canvas.create_rectangle(c1 * SIZE, r1 * SIZE, c1 * SIZE + SIZE, r1 * SIZE + SIZE,
                                         fill='blue', tags=('moves',))

    def run(self, auto):
        self.root.after(10, self.draw_grid)
        if auto and None not in (self.start, self.goal):
            self.root.after(50, self.model)
        self.root.mainloop()

    def draw_nums(self):
        for r in range(self.size):
            for c in range(self.size):
                i = r * self.size + c
                self.canvas.create_text((c + 0.5) * SIZE, (r + 0.5) * SIZE, text=str(i),
                                        font=('Arial', 20), fill='#444444', tags=('numbers', str(i)))

    def draw_grid(self):
        for pos in range(self.s2):
            if pos in self.blocks:
                color = BLOCK_COLOR
            elif pos == self.goal:
                color = GOAL_COLOR
            elif pos == self.start:
                color = START_COLOR
            else:
                color = BG

            x0, y0 = pos % self.size * SIZE, pos // self.size * SIZE
            self.canvas.create_rectangle(x0, y0, x0 + SIZE, y0 + SIZE,
                                         fill=color, width=WALL_WIDTH, tags=('rc_%s' % pos,), outline=GRID_COLOR)

        for a, b in self.walls:
            a_p, b_p = self.grid_bounding_flat(a), self.grid_bounding_flat(b)
            self.canvas.create_line(*(lambda x, y: x + y)(*tuple(set(a_p).intersection(b_p))),
                                    fill=WALL_COLOR, width=WALL_WIDTH, tags=(f'wall_{a}x{b}', f'wall_{b}x{a}'))

    def toggle_press(self, r, c):
        self.mouse_down = not self.mouse_down

        if self.mouse_down:
            self.mouse_start = r, c
        elif (r, c) == self.mouse_start:
            self.set_block(self.flatten(r, c))
        else:
            self.set_wall(self.flatten(self.mouse_start), self.flatten(r, c))

    def set_block(self, i):
        if i in (self.goal, self.start):
            return
        
        if i in self.legality.blocks:
            self.canvas.itemconfigure('rc_%s' % i, fill=BG)
            self.legality.blocks.remove(i)
            return

        self.canvas.itemconfigure(f'rc_%s' % i, fill=BLOCK_COLOR)
        self.legality.blocks.append(i)

    def iterate_pos(self, p):  # null -> start -> goal -> null
        if p in self.blocks:
            return

        color = START_COLOR
        if self.goal is None and p == self.start:
            self.start, self.goal, color = None, p, GOAL_COLOR
            self.legality.change_goal(p)
        elif p == self.goal:
            self.goal, color = None, BG
        elif self.start is None:
            self.start = p
        else:
            print('cannot have more than 1 start position at any given time')
            return

        self.canvas.itemconfigure('rc_%s' % p, fill=color)

    def set_wall(self, a, b):
        if (a, b) in self.legality.walls:
            self.legality.walls.remove((a, b))
            self.canvas.delete(f'wall_{a}x{b}')
            return
        elif (b, a) in self.legality.walls:
            self.legality.walls.remove((b, a))
            self.canvas.delete(f'wall_{b}x{a}')
            return

        self.legality.walls.append((a, b))

        a_p, b_p = self.grid_bounding_flat(a), self.grid_bounding_flat(b)
        intersection = tuple(set(a_p).intersection(b_p))

        if len(intersection) != 2:
            print(f'{a} and {b} were not adjacent')
            return

        p0, p1 = intersection
        self.canvas.create_line(*p0, *p1, fill=WALL_COLOR, width=WALL_WIDTH, tags=f'wall_{a}x{b}')

    def model(self):
        Q = np.zeros([self.s2, self.s2])
        states = [i for i in range(self.s2) if i not in self.blocks]

        l = int(log10(self.rounds)) + 1
        for rnd in range(self.rounds):
            if rnd % 100 == 99:
                print(f'\r{rnd + 1:>{l}} / {self.rounds}', end='')
                
            state = choice(states)
            action = choice(self.legality.moves(state))

            Q[state, action] += self.step * (self.legality[state, action] +
                                             self.decay * Q[action, np.argmax(Q[action])] - Q[state, action])

        print('\nfinished training')

        state, path = self.start, [self.start]
        for _ in range(self.s2):
            state = np.argmax(Q[state, ])
            path.append(state)
            if state == self.goal:  # goal
                break
        else:
            print('path not found', path)
            return

        self.path_display(path)

    def path_display(self, path):
        self.canvas.delete('path')

        points = [(lambda b: [sum(c) / 4 for c in b])(zip(*self.grid_bounding(*self.expand(point)))) for point in path]
        for i, point in enumerate(points[1:]):
            continue; self.canvas.create_line(*points[i], *points[i + 1], fill='blue', width=3, tags='path')

        Walk(self.canvas, points, time=3, radius=5)


class Walk:    
    def __init__(self, canvas, points, time, radius):
        self.points = np.asarray([np.array(point) for point in points])
            
        self.length = len(points)
        self.canvas = canvas
        self.total_time = time

        self.start = perf_counter()
        self.last = points[0]
        self.radius = radius
        self.drawing = canvas.create_oval(*(lambda x, y, r: (x - r, y - r, x + r, y + r))(*points[0], radius),
                                          fill=WALK_COLOR, tags=('Walker',))
        
        self.N = n * n
        
        turns, sign = 0, [False, False]
        for i, point in enumerate(self.points[1:]):
            new_sign = [a >= 0 for a in self.points[i] - point]
            if new_sign != sign:
                turns += 1
            sign = new_sign
        self.path = self.bspline(degree=turns)  # more turns, the higher the degree
            
        self.draw_point()
    
    def bspline(self, degree=3, periodic=False):
        return (lambda cv: (lambda degree, count: (lambda kv: np.array(interpolate.splev(np.linspace(periodic, (count - degree), self.N), (kv, cv.T, degree))).T)(np.arange(0 - degree, count + degree + degree - 1, dtype='int') if periodic else np.concatenate(([0] * degree, np.arange(count - degree + 1), [count - degree] * degree))))(np.clip(degree, 1, degree if periodic else self.length - 1), len(cv)))((lambda factor, fraction: np.concatenate((self.points,) * factor + (self.points[:fraction],)))(*divmod(self.length + degree + 1, self.length)) if periodic else self.points[:])
    
    def draw_point(self):
        t = min(1, (perf_counter() - self.start) / self.total_time)
        x, y = self.path[int((self.N - 1) * t)]
        
        self.canvas.moveto(self.drawing, x - self.radius, y - self.radius)
        self.canvas.create_line(*self.last, x, y, fill=WALK_COLOR, tags=('Walker',))
        self.last = x, y
        if t < self.total_time:
            self.canvas.after(1, self.draw_point)
        else:
            self.canvas.delete('Walker')


In [ ]:
import numpy as np
from random import choice, sample, randint
from tkinter import Tk, Canvas, StringVar, Button, Entry, Label
from math import dist
from time import perf_counter
from scipy.interpolate import BSpline


SIZE = 100
GOAL = 9

WALL_WIDTH = 1

BG = 'white'
BLOCK_COLOR = '#333333'
WALL_COLOR = BLOCK_COLOR
GOAL_COLOR = 'green'
START_COLOR = 'red'
GRID_COLOR = ''

WALK_COLOR = 'blue'
WALK_TIME = 3

ROUNDS = 10 ** 5
MAX_PLYS = 25
TRAINING_TYPE = 2


class App3:
    def flatten(self, r, c):
        return r * self.size + c

    def expand(self, k):
        return divmod(k, self.size)

    def bounding(self, *args):
        r, c = self.expand(args[0]) if len(args) == 1 else args
        return [((c + a) * SIZE, (r + b) * SIZE) for a in range(2) for b in range(2)]

    def intersection(self, a, b):
        return set(self.bounding(a)).intersection(self.bounding(b))

    def adjacent(self, k):
        r, c = divmod(k, self.size)
        if r != 0:
            yield k - self.size
        if r != self.size - 1:
            yield k + self.size
        if c != 0:
            yield k - 1
        if c != self.size - 1:
            yield k + 1
            
    @classmethod
    def random(cls, size, walls, blocks, start, goal):
        s2 = size * size

        def random_adjacent(k):
            choices = []
            r, c = divmod(k, size)
            if r != 0:
                choices.append(k - size)
            if r != size - 1:
                choices.append(k + size)
            if c != 0:
                choices.append(k - 1)
            if c != size - 1:
                choices.append(k + 1)
            return choice(choices)

        if start is None:
            start = 0
        elif start is True:
            start = randint(0, s2 - 1)

        if goal is None:
            goal = s2 - 1
        elif goal is True:
            goal = randint(0, s2 - 1)

        unique = [i for i in range(s2) if i not in (start, goal)]
        return cls(size, start=start, goal=goal,
                   blocks=sample(unique, int(blocks * s2)),
                   walls=[(k, random_adjacent(k)) for k in sample(unique, int(walls * s2))])

    def __init__(self, size, start=None, goal=None, blocks=None, walls=None):
        self.size = size

        self.start, self.goal = start, goal
        self.blocks, self.walls = blocks if blocks else [], walls if walls else []

        self.root, self.canvas, self.entry = self.create_window()
        self.mouse_down = False
        self.mouse_start = None

        self.showing_numbers = False
        self.showing_moves = False

    def run(self, auto=False):
        if auto:
            self.root.after(50, self.model)
        self.root.mainloop()

    def create_window(self):
        dim = SIZE * self.size

        root = Tk()
        root.geometry(f'{dim + SIZE}x{dim}')

        root.bind('<Return>', lambda *_: self.model())
        root.bind('<BackSpace>', lambda *_: self.canvas.delete('path'))

        canvas = Canvas(root, width=dim, height=dim, background=BG)
        canvas.grid(row=0, column=0, rowspan=5)

        root.bind('<KeyPress>', lambda e: self.key_handler(e.keysym, True))
        root.bind('<KeyRelease>', lambda e: self.key_handler(e.keysym, False))

        canvas.bind('<ButtonPress>', lambda e: self.mouse_handler(e.num, True, e.x, e.y))
        canvas.bind('<ButtonRelease>', lambda e: self.mouse_handler(e.num, False, e.x, e.y))
        canvas.bind('<Motion>', lambda e: self.reset_drawings())

        pos = y = 0
        for _ in range(self.size):
            y1, x = y + SIZE, 0
            for _ in range(self.size):
                color = BG
                if pos in self.blocks:
                    color = BLOCK_COLOR
                elif pos == self.start:
                    color = START_COLOR
                elif pos == self.goal:
                    color = GOAL_COLOR

                x1 = x + SIZE
                canvas.create_rectangle(x, y, x1, y1,
                                        fill=color, outline=GRID_COLOR, tags=(f'loc_{pos}',))
                x = x1
                pos += 1
            y = y1

        for a, b in self.walls:
            inter = self.intersection(a, b)
            if len(inter) != 2:
                print(f'{a} and {b} were not adjacent')
                continue
            a_p, b_p = inter
            canvas.create_line(*a_p, *b_p, fill=WALL_COLOR, width=WALL_WIDTH,
                               tags=(f'wall_{a}x{b}', f'wall_{b}x{a}'))

        entry = StringVar(root, '5')

        Label(root, text='rounds:').grid(row=1, column=1)
        Entry(root, textvariable=entry).grid(row=2, column=1)
        Button(root, text='run', command=self.model).grid(row=3, column=1)

        return root, canvas, entry

    def mouse_handler(self, button, boolean, x, y):
        pos = self.flatten(y // SIZE, x // SIZE)
        if button == 1:
            if boolean:
                self.mouse_start = pos
            elif pos == self.mouse_start:
                self.create_block(pos)
            else:
                self.create_wall(pos, self.mouse_start)
        elif button == 3:
            if boolean:
                self.iterate_pos(pos)

    def key_handler(self, key, boolean):
        if key == 'space':
            self.showing_numbers = boolean
            self.draw_nums() if boolean and self.showing_numbers else self.canvas.delete('numbers')

        elif key == 'm':
            self.showing_moves = boolean
            self.show_moves() if boolean else self.canvas.delete('moves')

    def get_moves(self, r, c):
        k = self.flatten(r, c)
        for move in list(self.adjacent(k)):
            if move in self.blocks:
                continue
            if (k, move) in self.walls or (move, k) in self.walls:
                continue
            yield move
        yield k
        
    def draw_nums(self):
        for r in range(self.size):
            for c in range(self.size):
                self.canvas.create_text((c + 0.5) * SIZE, (r + 0.5) * SIZE, text=str(r * self.size + c),
                                        font=('Arial', 20), fill='#444444', tags=('numbers',))

    def show_moves(self):
        r, c = (max(0, self.root.winfo_pointery() - self.root.winfo_rooty()) // SIZE,
                max(0, self.root.winfo_pointerx() - self.root.winfo_rootx()) // SIZE)
        for j in self.get_moves(r, c):
            r1, c1 = self.expand(j)
            self.canvas.create_rectangle(c1 * SIZE, r1 * SIZE, c1 * SIZE + SIZE, r1 * SIZE + SIZE,
                                         fill='blue', tags=('moves',))

    def reset_drawings(self):
        if self.showing_moves:
            self.canvas.delete('moves')
            self.show_moves()
            
    def change_color(self, k, color):
        self.canvas.itemconfigure(f'loc_{k}', fill=color)
        
    def iterate_pos(self, k):
        if k in self.blocks:
            return
        
        if k == self.start:
            if self.goal is not None:
                self.change_color(self.goal, BG)
            self.start, self.goal, color = None, self.start, GOAL_COLOR
        elif k == self.goal:
            self.goal, color = None, BG
        else:
            if self.start is not None:
                self.change_color(self.start, BG)
            self.start, color = k, START_COLOR
        self.change_color(k, color)
        
    def create_block(self, k):
        if k in (self.start, self.goal):
            return
        
        if k in self.blocks:
            self.blocks.remove(k)
            self.change_color(k, BG)
            return
    
        self.change_color(k, BLOCK_COLOR)
        self.blocks.append(k)
        
    def create_wall(self, a, b):
        if (a, b) in self.walls:
            self.walls.remove((a, b))
            self.canvas.delete(f'wall_{a}x{b}')
            return
        elif (b, a) in self.walls:
            self.walls.remove((b, a))
            self.canvas.delete(f'wall_{b}x{a}')
            return

        inter = self.intersection(a, b)
        
        if len(inter) != 2:
            print(f'{a} and {b} were not adjacent')
            return
        
        a_p, b_p = inter
        self.canvas.create_line(*a_p, *b_p, fill=WALL_COLOR, width=WALL_WIDTH,
                                tags=(f'wall_{a}x{b}', f'wall_{b}x{a}'))
        self.walls.append((a, b))
            
    def model(self):
        if None in (self.start, self.goal):
            print(f'None appears >= 1 times in {(self.start, self.goal)}, both must be present to train the model')
            return
        
        rounds = int((lambda a, b: 10 ** float(a) if a.isdecimal() else b)(self.entry.get(), ROUNDS))
        model = PathFinderModel(self.size, self.walls, self.blocks, self.start, self.goal, training_type=TRAINING_TYPE)
        model.train(rounds)

        self.draw_path(model.play_game())

    def draw_path(self, path):
        self.canvas.delete('path')

        print(path)
        centers = np.array([np.array([sum(c) / 4 for c in zip(*self.bounding(k))]) for k in path])

        PathDisplay(centers, self.canvas, 'linear')
        PathDisplay(centers, self.canvas, 'interpolated', definition=self.size * self.size)


class PathDisplay:
    @staticmethod
    def circle(x, y, r):
        return x - r, y - r, x + r, y + r

    def __init__(self, points, canvas, style='linear', definition=100, radius=0.05):
        self.points = points if style == 'linear' else (self.b_spline(points, definition) if style == 'interpolated' else [])
        self.length = len(self.points)
        self.canvas = canvas

        self.last = self.points[0]
        self.radius = int(radius * SIZE)

        self.ms = int(1000 * (WALK_TIME / self.length) - 1)
        self.start = perf_counter()
        self.drawing = canvas.create_oval(*self.circle(*self.last, self.radius), fill=WALK_COLOR, tags=('path',))
        self.iterate()

    def b_spline(self, points, n):
        degree, sign = 0, [False, False]
        for i, point in enumerate(points[1:]):
            new_sign = [a >= 0 for a in points[i] - point]
            if new_sign != sign:
                degree += 1
            sign = new_sign

        count = points.shape[0]

        degree = np.clip(degree, 1, count - 1)
        kv = np.clip(np.arange(count + degree + 1) - degree, 0, count - degree)

        return BSpline(kv, points, degree)(np.linspace(0, count - degree, n))

    def iterate(self):
        t = (perf_counter() - self.start) / WALK_TIME
        if t >= 1:
            return

        x, y = self.points[int(self.length * t)]
        self.canvas.create_line(*self.last, x, y, fill=WALK_COLOR, tags=('path',))
        self.canvas.moveto(self.drawing, x - self.radius, y - self.radius)
        self.last = x, y

        self.canvas.after(self.ms, self.iterate)


In [18]:
a = App3(4)
a.run()

      9999[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]


In [22]:
class Model:
    step_size = 1
    decay = 0.75

    def __init__(self, total_states, total_actions, available_states, legality, start, goal,
                 record_intervals=0, training_type=1):
        self.legality = legality

        self.states = available_states
        self.states_n, self.actions_n = total_states, total_actions
        self.Q = np.zeros([self.states_n, self.actions_n])

        self.start, self.goal = start, goal
        self.state = start

        self.record = {}
        self.record_training = max(0, record_intervals)

        self.train = {1: self.train_A,
                      2: self.train_B}[training_type]

    def step(self, action):
        legal = self.legality(self.state, action)
        if legal:
            self.state = action

        return legal, (action == self.goal) and legal

    def reset(self):
        self.state = self.start

    def play_game(self):
        self.reset()

        record = [self.state]
        for _ in range(MAX_PLYS):
            _, finished = self.step(np.argmax(self.Q[self.state, ]))
            record.append(self.state)
            if finished:
                return record
        return record

    def train_A(self, rounds):
        for rnd in range(rounds):
            self.reset()
            for current_turn in range(MAX_PLYS):  # play game
                action = np.argmax(self.Q[self.state, ] + np.random.randn(1, self.actions_n) / (rnd + 1))
                legal, finished = self.step(action)

                self.Q[self.state, action] += self.step_size * (legal + self.decay * np.max(self.Q[action])
                                                                - self.Q[self.state, action])

                if finished:
                    print(f'finished round {rnd} in {current_turn} turns')
                    break

            if self.record_training != 0 and rnd % self.record_training == 0:
                self.record[rnd] = self.play_game()

    def train_B(self, rounds):
        actions = {state: [i for i in self.states if self.legality(state, i)] for state in self.states}
        for rnd in range(rounds):
            print(f'\r{rnd:>10}', end='')
            state = choice(self.states)
            action = choice(actions[state])

            self.Q[state, action] += self.step_size * (1 + self.decay * np.max(self.Q[action]) - self.Q[state, action])

        print([list(a) for a in self.Q])

        
class PathFinderModel(Model):
    def __init__(self, n, walls, blocks, start, goal, record_interval=0, training_type=1):
        self.size, self.row = n * n, n

        self.blocks, self.walls = blocks, walls
        self.start, self.goal = start, goal

        self.states = [i for i in range(self.size) if i not in self.blocks]

        super().__init__(self.size, self.size, self.states, self.legality, start, goal,
                         record_interval, training_type=training_type)

    def expand(self, k):
        return k // self.row, k % self.row

    def legality(self, state, action):
        if state == action:
            return False

        if action in self.blocks:
            return False

        if (state, action) in self.walls or (action, state) in self.walls:
            return False

        return dist(self.expand(state), self.expand(action)) <= 1.0

In [23]:
App3(4).run()

     99999[[0.0, 3.999999999999999, 0.0, 0.0, 3.999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.999999999999999, 0.0, 3.999999999999999, 0.0, 0.0, 3.999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 3.999999999999999, 0.0, 3.999999999999999, 0.0, 0.0, 3.999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.999999999999999, 0.0, 0.0, 0.0, 0.0, 3.999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.999999999999999, 0.0, 0.0, 0.0, 0.0, 3.999999999999999, 0.0, 0.0, 3.999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 3.999999999999999, 0.0, 0.0, 3.999999999999999, 0.0, 3.999999999999999, 0.0, 0.0, 3.999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 3.999999999999999, 0.0, 0.0, 3.999999999999999, 0.0, 3.999999999999999, 0.0, 0.0, 3.999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 3.999999999999999, 0.0, 0.0, 3.999999999999999, 0.0, 0.0, 0.0, 0.0, 3.999999999999999, 0.0, 0.0, 0.0, 0.0

In [ ]:
def model(self):
    Q = np.zeros([self.s2, self.s2])
    states = [i for i in range(self.s2) if i not in self.blocks]

    l = int(log10(self.rounds)) + 1
    for rnd in range(self.rounds):
        if rnd % 100 == 99:
            print(f'\r{rnd + 1:>{l}} / {self.rounds}', end='')

        state = choice(states)
        action = choice(self.legality.moves(state))

        Q[state, action] += self.step * (self.legality[state, action] +
                                         self.decay * Q[action, np.argmax(Q[action])] - Q[state, action])

    print('\nfinished training')

    state, path = self.start, [self.start]
    for _ in range(self.s2):
        state = np.argmax(Q[state, ])
        path.append(state)
        if state == self.goal:  # goal
            break
    else:
        print('path not found', path)
        return

    self.path_display(path)

In [15]:
n, r = 5, 4

def default(walls=False):
    w = [(1, 2), (1, 9), (3, 11), (5, 6), (5, 13), (7, 15), (8, 16), (9, 10), (10, 18), (11, 12), (12, 20), (13, 14), (14, 22), (17, 18), (17, 25), (19, 20), (19, 27), (21, 22), (21, 29), (23, 31), (24, 32), (25, 26), (26, 34), (27, 28), (28, 36), (29, 30), (30, 38), (33, 41), (34, 33), (35, 36), (35, 43), (37, 38), (37, 45), (39, 47), (40, 48), (41, 42), (42, 50), (43, 44), (44, 52), (45, 46), (46, 54), (49, 50), (49, 57), (51, 52), (51, 59), (53, 54), (53, 61), (55, 63), (59, 60)]
    app = App2(n, start=0, goal=7 if walls else n * n - 1, walls=w if walls else None, rounds=10 ** r)
    return app
    
def random():
    app = App2.random_start(n, walls=0.5, rounds=10 ** r)
    return app

In [16]:
default().run(False)

10000 / 10000
finished training
